In [3]:
!pip install sound

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 2.9 MB/s eta 0:00:00


In [8]:
!pip install pyglet

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.1 MB/s eta 0:00:00


In [ ]:
# smart_solar_ai.py
import tkinter as tk
import time
import math
import random

# -------------------
# CONFIG
# -------------------
DAY_SECONDS = 30            # daytime duration
NIGHT_SECONDS = 30          # nighttime duration
CYCLE_SECONDS = DAY_SECONDS + NIGHT_SECONDS
UPDATE_MS = 120             # UI update interval in milliseconds (~8 FPS)
BATTERY_CHARGE_MAX = 0.8    # max battery %/s when sun is strongest (scaled by solar intensity)
BATTERY_DRAIN_PUMP = 0.6    # battery %/s while pumping
TANK_USE_DAY = 0.15         # tank %/s used during day
TANK_FILL_PUMP = 0.8        # tank %/s filled while pumping at night
MIN_BATTERY_FOR_PUMP = 20.0 # minimum battery % to allow pumping
FULL = 100.0

# -------------------
# STATE
# -------------------
solar_pct = 0.0
battery_pct = 60.0
tank_pct = 50.0
is_day = True
ai_message = "System starting..."
ai_text_full = ""
ai_typing_index = 0
ai_is_typing = False

start_time = time.perf_counter()

# -------------------
# UI
# -------------------
root = tk.Tk()
root.title("Smart Solar Water System — AI Dashboard")
root.geometry("920x520")
root.resizable(False, False)

# -- top title
title = tk.Label(root, text="SMART SOLAR WATER SYSTEM  —  AI ASSISTANT",
                 font=("Segoe UI", 16, "bold"), bg="#0b1013", fg="#9be7ff", pady=8)
title.pack(fill="x")

main_frame = tk.Frame(root, bg="#0b1013")
main_frame.pack(fill="both", expand=True, padx=12, pady=12)

# Left column: Solar (box), Sun/Moon box, small status
left_col = tk.Frame(main_frame, bg="#0b1013")
left_col.grid(row=0, column=0, sticky="n", padx=(0,12))

# Solar box (with sun/moon animation inside)
solar_box = tk.Frame(left_col, bg="#0f2630", bd=2, relief="ridge", width=300, height=200)
solar_box.pack_propagate(False)
solar_box.pack()

solar_title = tk.Label(solar_box, text="Solar Panel", font=("Segoe UI", 11, "bold"),
                       bg="#0f2630", fg="white")
solar_title.pack(anchor="n", pady=(8,0))

# Canvas inside solar box for moving sun/moon and solar percent
solar_canvas = tk.Canvas(solar_box, width=260, height=120, bg="#071019", highlightthickness=0)
solar_canvas.pack(pady=8)
# sun and moon drawn as ovals; we'll move/resize them
sun_id = solar_canvas.create_oval(0,0,0,0, fill="#ffd24d", outline="")
moon_id = solar_canvas.create_oval(0,0,0,0, fill="#e7f0ff", outline="", state="hidden")
solar_pct_text = tk.Label(solar_box, text="Solar: 0%", font=("Segoe UI", 10), bg="#0f2630", fg="#aee7ff")
solar_pct_text.pack(anchor="s", pady=(0,8))

# Middle column: Battery box and Pump indicator
mid_col = tk.Frame(main_frame, bg="#0b1013")
mid_col.grid(row=0, column=1, sticky="n", padx=(0,12))

battery_box = tk.Frame(mid_col, bg="#0f2630", bd=2, relief="ridge", width=260, height=120)
battery_box.pack_propagate(False)
battery_box.pack()
battery_title = tk.Label(battery_box, text="Battery", font=("Segoe UI", 11, "bold"),
                         bg="#0f2630", fg="white")
battery_title.pack(anchor="n", pady=(8,0))

battery_canvas = tk.Canvas(battery_box, width=220, height=40, bg="#071019", highlightthickness=0)
battery_canvas.pack(pady=12)
battery_text_label = tk.Label(battery_box, text="Battery: 60%", font=("Segoe UI", 10), bg="#0f2630", fg="#aee7ff")
battery_text_label.pack()

pump_box = tk.Frame(mid_col, bg="#0b1013")
pump_box.pack(pady=16)
pump_status_label = tk.Label(pump_box, text="Pump: OFF", font=("Segoe UI", 14, "bold"),
                             bg="#0b1013", fg="#ff7b7b")
pump_status_label.pack()

# Right column: Water tank and Ground water (animated)
right_col = tk.Frame(main_frame, bg="#0b1013")
right_col.grid(row=0, column=2, sticky="n")

tank_box = tk.Frame(right_col, bg="#0f2630", bd=2, relief="ridge", width=300, height=200)
tank_box.pack_propagate(False)
tank_box.pack()

tank_title = tk.Label(tank_box, text="Water Tank", font=("Segoe UI", 11, "bold"),
                      bg="#0f2630", fg="white")
tank_title.pack(anchor="n", pady=(8,0))

tank_canvas = tk.Canvas(tank_box, width=260, height=120, bg="#071019", highlightthickness=0)
tank_canvas.pack(pady=8)
tank_pct_text = tk.Label(tank_box, text="Tank: 50%", font=("Segoe UI", 10), bg="#0f2630", fg="#aee7ff")
tank_pct_text.pack(anchor="s", pady=(0,8))

# Bottom: AI panel (typewriter) — boxed
ai_frame = tk.Frame(root, bg="#071019", bd=2, relief="ridge")
ai_frame.pack(fill="x", padx=12, pady=(6,12))
ai_title = tk.Label(ai_frame, text="AI ASSISTANT", font=("Segoe UI", 11, "bold"), bg="#071019", fg="#9be7ff")
ai_title.pack(anchor="w", padx=8, pady=(6,0))

ai_display = tk.Label(ai_frame, text="", font=("Segoe UI", 12), bg="#071019", fg="#dff6ff", justify="left", anchor="w", wraplength=880)
ai_display.pack(fill="x", padx=8, pady=(6,10))

# small clock/time and mode label
mode_label = tk.Label(root, text="Mode: --", font=("Segoe UI", 10, "italic"), bg="#0b1013", fg="#cfefff")
mode_label.place(x=12, y=42)

# -------------------
# HELPERS
# -------------------
def get_cycle_progress():
    """Return a tuple (is_day, progress) where progress is 0..1 inside current half-cycle."""
    elapsed = (time.perf_counter() - start_time) % CYCLE_SECONDS
    if elapsed < DAY_SECONDS:
        return True, elapsed / DAY_SECONDS
    else:
        return False, (elapsed - DAY_SECONDS) / NIGHT_SECONDS

def lerp_color(c1, c2, t):
    """Linear interpolate between two hex colors c1, c2 with t in [0,1]."""
    c1 = c1.lstrip("#"); c2 = c2.lstrip("#")
    r1, g1, b1 = int(c1[0:2],16), int(c1[2:4],16), int(c1[4:6],16)
    r2, g2, b2 = int(c2[0:2],16), int(c2[2:4],16), int(c2[4:6],16)
    r = int(r1 + (r2-r1)*t); g = int(g1 + (g2-g1)*t); b = int(b1 + (b2-b1)*t)
    return f"#{r:02x}{g:02x}{b:02x}"

def set_ai_message(new_msg):
    """Set AI full message and start typewriter reveal."""
    global ai_message, ai_text_full, ai_typing_index, ai_is_typing
    ai_text_full = f"AI: {new_msg}"
    ai_typing_index = 0
    ai_is_typing = True
    ai_display.config(text="")  # clear immediate
    typewriter_step()

def typewriter_step():
    """Reveal the AI message one character at a time."""
    global ai_typing_index, ai_is_typing
    if ai_typing_index < len(ai_text_full):
        ai_display.config(text=ai_text_full[:ai_typing_index+1])
        ai_typing_index += 1
        # typing speed varies a bit
        root.after(28, typewriter_step)
    else:
        ai_is_typing = False

# -------------------
# LOGIC / AI BRAIN
# -------------------
def ai_logic(is_day_now, prog):
    """Decide action and set ai message; also update system dynamics for 1 tick."""
    global solar_pct, battery_pct, tank_pct

    # SUNLIGHT shape: use sine shape during day (0..1..0) for intensity
    if is_day_now:
        sunlight_factor = math.sin(prog * math.pi)  # peaks at midday
        solar_pct = sunlight_factor * 100.0
        # battery charges proportional to sunlight
        battery_pct += BATTERY_CHARGE_MAX * sunlight_factor * (UPDATE_MS/1000.0)
        battery_pct = min(battery_pct, FULL)
        # water usage during day
        tank_pct -= TANK_USE_DAY * (UPDATE_MS/1000.0)
        tank_pct = max(0.0, tank_pct)
        # decide AI comment
        if battery_pct >= 99.0:
            set_ai_message("Battery is fully charged. Holding pump until needed.")
        else:
            set_ai_message(f"Charging from sun — solar intensity {int(solar_pct)}%. Battery rising.")
    else:
        # night
        solar_pct = 0.0
        # decide whether to pump: prefer pumping when battery healthy and tank not full
        if tank_pct < 95.0 and battery_pct > MIN_BATTERY_FOR_PUMP:
            # pump on: fills tank, drains battery
            tank_pct += TANK_FILL_PUMP * (UPDATE_MS/1000.0)
            battery_pct -= BATTERY_DRAIN_PUMP * (UPDATE_MS/1000.0)
            battery_pct = max(0.0, battery_pct)
            set_ai_message("Using stored energy to pump water to tank.")
        else:
            # conserve battery
            battery_pct -= (BATTERY_CHARGE_MAX * 0.12) * (UPDATE_MS/1000.0)  # small passive drain
            battery_pct = max(0.0, battery_pct)
            if battery_pct <= MIN_BATTERY_FOR_PUMP:
                set_ai_message("Battery low — conserving energy. Pump idle.")
            else:
                set_ai_message("Night: Battery sufficient but tank nearly full; holding pump.")

    # clamp
    battery_pct = max(0.0, min(FULL, battery_pct))
    tank_pct = max(0.0, min(FULL, tank_pct))

# -------------------
# UI UPDATE
# -------------------
# Positions for sun/moon movement inside solar_canvas
inner_w = 260
inner_h = 120
sun_radius = 20
moon_radius = 18

def update_ui():
    global is_day, solar_pct, battery_pct, tank_pct

    is_day_now, prog = get_cycle_progress()
    is_day = is_day_now

    # animate background fade between day_color and night_color of main root/per-box
    day_color = "#bfe9ff"   # light sky
    night_color = "#031024" # deep night
    # compute blend: day -> night across full cycle with smoother transitions
    overall_elapsed = (time.perf_counter() - start_time) % CYCLE_SECONDS
    blend_t = (overall_elapsed / CYCLE_SECONDS)  # 0..1
    # We want stronger day in first half, night in second; map to smooth step
    if overall_elapsed < DAY_SECONDS:
        # day-to-dawn
        t = overall_elapsed / DAY_SECONDS
        bg = lerp_color(day_color, "#d4f1ff", t*0.1)  # slight change during day
    else:
        t = (overall_elapsed - DAY_SECONDS) / NIGHT_SECONDS
        bg = lerp_color("#bfe9ff", night_color, t)

    # apply background to frames
    root.configure(bg=bg)
    main_frame.configure(bg=bg)
    # set mode label
    mode_label.config(text=f"Mode: {'Day' if is_day else 'Night'}")

    # update AI + logic
    ai_logic(is_day, prog)

    # draw solar sun/moon inside solar_canvas
    solar_canvas.delete("all")
    if is_day:
        # sun visible, move left->right across inner width using prog
        x_center = 20 + prog * (inner_w - 40)
        y_center = inner_h/2 - math.sin(prog * math.pi) * 18
        sun_x1, sun_y1 = x_center - sun_radius, y_center - sun_radius
        sun_x2, sun_y2 = x_center + sun_radius, y_center + sun_radius
        # draw gentle glow ring
        solar_canvas.create_oval(sun_x1-8, sun_y1-8, sun_x2+8, sun_y2+8, fill="#fff2cc", outline="", stipple="")
        solar_canvas.create_oval(sun_x1, sun_y1, sun_x2, sun_y2, fill="#ffd24d", outline="")
        # solar text
        solar_pct_text.config(text=f"Solar: {int(solar_pct)}%")
    else:
        # moon visible, moves right->left (use prog 0..1 across night; we invert so it moves right->left)
        x_center = 20 + (1-prog) * (inner_w - 40)
        y_center = inner_h/2 - math.sin(prog * math.pi) * 10
        mx1, my1 = x_center - moon_radius, y_center - moon_radius
        mx2, my2 = x_center + moon_radius, y_center + moon_radius
        # moon glow
        solar_canvas.create_oval(mx1-6, my1-6, mx2+6, my2+6, fill="#f1f8ff", outline="")
        solar_canvas.create_oval(mx1, my1, mx2, my2, fill="#e7f0ff", outline="")
        solar_pct_text.config(text=f"Solar: {int(solar_pct)}% (Night)")

    # update battery canvas
    battery_canvas_w = 220
    battery_canvas_h = 40
    battery_canvas.delete("all")
    # background bar
    battery_canvas.create_rectangle(2, 2, battery_canvas_w-2, battery_canvas_h-2, fill="#071019", outline="#2b2f35")
    # fill length
    fill_len = (battery_pct / 100.0) * (battery_canvas_w-6)
    # color gradient based on battery
    if battery_pct >= 70:
        bcolor = "#4caf50"
    elif battery_pct >= 30:
        bcolor = "#ffb74d"
    else:
        bcolor = "#e53935"
    battery_canvas.create_rectangle(4, 4, 4 + fill_len, battery_canvas_h-4, fill=bcolor, outline="")
    battery_text_label.config(text=f"Battery: {battery_pct:.1f}%")

    # update tank canvas
    tank_canvas.delete("all")
    tank_canvas.create_rectangle(2, 2, inner_w-2, inner_h-2, fill="#071019", outline="#2b2f35")
    tank_fill_len = (tank_pct / 100.0) * (inner_w-6)
    tank_canvas.create_rectangle(4, inner_h-6, 4 + tank_fill_len, inner_h-4, fill="#00aaff", outline="")
    tank_pct_text.config(text=f"Tank: {tank_pct:.1f}%")

    # pump status (derived)
    pump_on_now = (not is_day) and (tank_pct < 98.5) and (battery_pct > MIN_BATTERY_FOR_PUMP)
    pump_status_label.config(text=f"Pump: {'ON' if pump_on_now else 'OFF'}", fg="#7cff7c" if pump_on_now else "#ff7b7b")

    # update solar pct label
    # (already updated earlier)

    # schedule next update
    root.after(UPDATE_MS, update_ui)

# -------------------
# START
# -------------------
# kick off initial AI message
set_ai_message("Initializing system and calibrating sensors...")

# start UI loop
root.after(UPDATE_MS, update_ui)
root.mainloop()